In [1]:
import csv
import cv2
import itertools
import numpy as np
import pandas as pd
import os
import sys
import tempfile
import tqdm
train = pd.read_csv('/kaggle/input/ai2022-competition/train.csv')

# Using a Custom Dataset for Training the MoveNet Preprocessor
We used the AII2022 Conference Competition Data on Kaggle for this.

In [2]:
train = pd.read_csv('/kaggle/input/ai2022-competition/train.csv')

In [3]:
train.head()

,ID,PoseLabel
0,00018.jpg,0_1_77
1,00020.jpg,1_5_72
2,00021.jpg,1_5_30
3,00028.jpg,0_3_73
4,00030.jpg,4_14_10


In [4]:
test = pd.read_csv('/kaggle/input/ai2022-competition/test.csv')

In [5]:
test.head()

,ID
0,74866.jpg
1,74868.jpg
2,74873.jpg
3,74882.jpg
4,74885.jpg


In [6]:
os.makedirs('sorted_train')

In [7]:
import shutil

# Define paths
csv_file_path = '/kaggle/input/ai2022-competition/train.csv'
images_dirs = {
    'train_1': '/kaggle/input/ai2022-competition/Train_part_1/Train_part_1',
    'train_2': '/kaggle/input/ai2022-competition/Train_part_2/Train_part_2',
    'train_3': '/kaggle/input/ai2022-competition/Train_part_3/Train_part_3',
}
output_dir = '/kaggle/working/sorted_train'

# Read CSV file into a pandas DataFrame
data = pd.read_csv(csv_file_path)

# Loop through each row in the DataFrame
for index, row in data.iterrows():
    image_filename = row['ID']
    label = row['PoseLabel']

    # Determine which dataset (folder) the image belongs toa
    for dataset, images_dir in images_dirs.items():
        image_src_path = os.path.join(images_dir, image_filename)
        if os.path.exists(image_src_path):
            # Create a folder for the label if it doesn't exista
            label_folder = os.path.join(output_dir, label)
            if not os.path.exists(label_folder):
                os.makedirs(label_folder)

            # Move the image to the corresponding folder
            image_dest_path = os.path.join(label_folder, image_filename)
            shutil.copy(image_src_path, image_dest_path)
            break

print("Images sorted into separate folders based on labels and datasets.")



Images sorted into separate folders based on labels and datasets.


In [8]:
train['PoseLabel'].nunique()

80

In [9]:
os.listdir('/kaggle/input/ai2022-competition')

['sample_submission.csv',
 'Train_part_3',
 'Train_part_1',
 'Train_part_2',
 'train.csv',
 'test.csv',
 'Test',
 'ReadMe.pdf']

In [10]:
len(next(os.walk('sorted_train'))[1])

80

In [11]:
os.listdir('sorted_train')

['5_17_70',
 '5_19_2',
 '4_13_48',
 '2_10_19',
 '1_8_35',
 '0_2_75',
 '4_16_26',
 '0_1_16',
 '4_13_33',
 '1_4_59',
 '4_14_38',
 '2_10_46',
 '1_7_67',
 '1_7_52',
 '1_4_28',
 '0_1_77',
 '5_17_5',
 '4_14_10',
 '1_4_41',
 '0_2_40',
 '0_2_31',
 '2_9_11',
 '4_13_66',
 '0_0_8',
 '4_16_44',
 '2_9_14',
 '0_1_60',
 '3_12_51',
 '2_10_55',
 '2_9_50',
 '0_0_18',
 '0_1_17',
 '4_14_20',
 '0_2_22',
 '3_12_45',
 '1_4_57',
 '1_8_0',
 '3_11_23',
 '0_3_73',
 '0_3_61',
 '4_13_79',
 '4_13_25',
 '2_9_53',
 '5_17_78',
 '4_14_27',
 '1_5_72',
 '1_8_47',
 '4_15_54',
 '5_19_4',
 '0_2_21',
 '0_3_39',
 '5_17_69',
 '2_9_24',
 '4_13_65',
 '1_4_3',
 '0_3_62',
 '1_5_13',
 '4_15_56',
 '5_17_71',
 '1_6_58',
 '1_7_49',
 '1_6_76',
 '3_11_63',
 '4_16_42',
 '4_14_9',
 '3_11_32',
 '0_1_36',
 '5_17_6',
 '1_5_1',
 '0_0_68',
 '1_5_30',
 '3_11_64',
 '4_13_12',
 '1_7_34',
 '0_3_74',
 '3_11_37',
 '0_2_29',
 '5_18_7',
 '4_16_15',
 '5_17_43']

### Thus we have sorted our train data to be organised by PoseLabel which is required for the MoveNet Preprocessor to work successfully

# Splitting the train data into Train and Test

In [12]:
use_custom_dataset = True

dataset_is_split = False

In [13]:
import os
import random
import shutil

def split_into_train_test(images_origin, images_dest, test_split):
  """Splits a directory of sorted images into training and test sets.

  Args:
    images_origin: Path to the directory with your images. This directory
      must include subdirectories for each of your labeled classes. For example:
      yoga_poses/
      |__ downdog/
          |______ 00000128.jpg
          |______ 00000181.jpg
          |______ ...
      |__ goddess/
          |______ 00000243.jpg
          |______ 00000306.jpg
          |______ ...
      ...
    images_dest: Path to a directory where you want the split dataset to be
      saved. The results looks like this:
      split_yoga_poses/
      |__ train/
          |__ downdog/
              |______ 00000128.jpg
              |______ ...
      |__ test/
          |__ downdog/
              |______ 00000181.jpg
              |______ ...
    test_split: Fraction of data to reserve for test (float between 0 and 1).
  """
  _, dirs, _ = next(os.walk(images_origin))

  TRAIN_DIR = os.path.join(images_dest, 'train')
  TEST_DIR = os.path.join(images_dest, 'test')
  os.makedirs(TRAIN_DIR, exist_ok=True)
  os.makedirs(TEST_DIR, exist_ok=True)

  for dir in dirs:
    # Get all filenames for this dir, filtered by filetype
    filenames = os.listdir(os.path.join(images_origin, dir))
    filenames = [os.path.join(images_origin, dir, f) for f in filenames if (
        f.endswith('.png') or f.endswith('.jpg') or f.endswith('.jpeg') or f.endswith('.bmp'))]
    # Shuffle the files, deterministically
    filenames.sort()
    random.seed(42)
    random.shuffle(filenames)
    # Divide them into train/test dirs
    os.makedirs(os.path.join(TEST_DIR, dir), exist_ok=True)
    os.makedirs(os.path.join(TRAIN_DIR, dir), exist_ok=True)
    test_count = int(len(filenames) * test_split)
    for i, file in enumerate(filenames):
      if i < test_count:
        destination = os.path.join(TEST_DIR, dir, os.path.split(file)[1])
      else:
        destination = os.path.join(TRAIN_DIR, dir, os.path.split(file)[1])
      shutil.copyfile(file, destination)
    print(f'Moved {test_count} of {len(filenames)} from class "{dir}" into test.')
  print(f'Your split dataset is in "{images_dest}"')


In [14]:
dataset_in = '/kaggle/working/sorted_train'

# You can leave the rest alone:
if not os.path.isdir(dataset_in):
  raise Exception("dataset_in is not a valid directory")
if dataset_is_split:
  IMAGES_ROOT = dataset_in
else:
  dataset_out = '/kaggle/working/split_data'
  split_into_train_test(dataset_in, dataset_out, test_split=0.2)
  IMAGES_ROOT = dataset_out



Moved 9 of 46 from class "5_17_70" into test.
Moved 54 of 273 from class "5_19_2" into test.
Moved 20 of 102 from class "4_13_48" into test.
Moved 25 of 127 from class "2_10_19" into test.
Moved 17 of 89 from class "1_8_35" into test.
Moved 24 of 121 from class "0_2_75" into test.
Moved 29 of 147 from class "4_16_26" into test.
Moved 10 of 50 from class "0_1_16" into test.
Moved 23 of 117 from class "4_13_33" into test.
Moved 12 of 60 from class "1_4_59" into test.
Moved 33 of 167 from class "4_14_38" into test.
Moved 20 of 104 from class "2_10_46" into test.
Moved 17 of 89 from class "1_7_67" into test.
Moved 50 of 251 from class "1_7_52" into test.
Moved 30 of 152 from class "1_4_28" into test.
Moved 36 of 182 from class "0_1_77" into test.
Moved 36 of 182 from class "5_17_5" into test.
Moved 109 of 548 from class "4_14_10" into test.
Moved 6 of 30 from class "1_4_41" into test.
Moved 32 of 161 from class "0_2_40" into test.
Moved 33 of 169 from class "0_2_31" into test.
Moved 18 of 

## Now let's download the train and test folders to further use them for preprocessing

In [15]:
import zipfile

# Define the path of the folder you want to download
folder_path = '/kaggle/working/split_data'

# Define the path of the .zip file you want to create
zip_file_path = '/kaggle/working/split_data.zip'

# Create a .zip file containing the contents of the folder
with zipfile.ZipFile(zip_file_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, _, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            arcname = os.path.relpath(file_path, folder_path)
            zipf.write(file_path, arcname)

print(f"Folder '{folder_path}' has been compressed and saved as '{zip_file_path}'.")

Folder '/kaggle/working/split_data' has been compressed and saved as '/kaggle/working/split_data.zip'.


In [16]:
import IPython
IPython.display.FileLink('split_data.zip')


/kaggle/working/split_data.zip

# Next Steps
After this we downloaded the split train data and split it into two and preprocessed them separately because of RAM limits on Kaggle, and then preprocessed the test data as well.